In [37]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

In [73]:
urls_wc18 = "https://www.betexplorer.com/soccer/world/world-cup-2018/"
urls_wc14 = "https://www.betexplorer.com/soccer/world/world-cup-2014/"
urls_wc10 = "https://www.betexplorer.com/soccer/world/world-cup-2010/"
urls_wc06 = "https://www.betexplorer.com/soccer/world/world-cup-2006/"
urls_wc02 = "https://www.betexplorer.com/soccer/world/world-cup-2002/"


In [63]:
pages = [r.get(url_wc18, params={"stage": "OneVXSrp"}).text, r.get(url_wc18).text]

In [64]:
pages

['<!DOCTYPE html>\n<!--[if lt IE 8 ]><html lang="cs" class="no-js oldie ie7 no-svg no-transform no-boxshadow no-css-animations"><![endif]-->\n<!--[if IE 8 ]><html lang="cs" class="no-js oldie no-svg no-transform no-boxshadow no-css-animations"><![endif]-->\n<!--[if IE 9 ]><html lang="cs" class="no-js ie9"><![endif]-->\n<!--[if (gt IE 9)|!(IE)]><!--><html lang="cs" class="no-js"><!--<![endif]-->\n<head>\n\t<meta charset="utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta name="viewport" id="viewportmeta" content="width=device-width, initial-scale=1.0">\n\t<meta http-equiv="expires" content="86400">\n\t<meta name="robots" content="all,follow">\n\t<meta name="keywords" content="World Cup 2018, Soccer, sports statistics, fixtures, results, odds, tables, sports stats, betting on sports, soccer, hockey, odds comparison">\n\t<meta name="description" content="World Cup 2018 results, tables, fixtures, and other stats for World Cup 2018. Compare teams, find the best odds a

In [65]:
def extract_data(info):
    
    team_home = info[0].text.split("-")[0][:-1]
    team_away = info[0].text.split("-")[-1][1:]

    goals_home = int(info[1].text.split(":")[0])
    goals_away = int(info[1].text.split(":")[-1].split(" ")[0])

    try:
        odds_home = info[2].span.span.span.attrs["data-odd"]
    except: 
        odds_home = info[2].attrs["data-odd"]

    try:
        odds_draw = info[3].span.span.span.attrs["data-odd"]
    except:
        odds_draw = info[3].attrs["data-odd"]

    try:
        odds_away = info[4].span.span.span.attrs["data-odd"]
    except:
        odds_away = info[4].attrs["data-odd"]

    date = info[5].text

    match_info = (team_home, team_away, goals_home, goals_away, odds_home, odds_draw, odds_away, date)

    return match_info

In [69]:
def create_dataframe(pages):
    all_matches_info = []

    for page in pages:
        soup = BeautifulSoup(page, 'html.parser')
        name_box = soup.find_all("table")
        data = name_box[0].contents

        for d in data:
            if d != "\n" and d.findAll("td"):
                #print(d.tr)
                all_matches_info.append(extract_data(d.findAll("td")))

    df_group = pd.DataFrame(all_matches_info, columns=["team_home","team_away","goals_home","goals_away","odds_home",
                                                       "odds_draw","odds_away","date"])
    
    return df_group

In [70]:
create_dataframe(pages)

,team_home,team_away,goals_home,goals_away,odds_home,odds_draw,odds_away,date
0,Japan,Poland,0,1,2.91,3.13,2.67,28.06.2018
1,Senegal,Colombia,0,1,5.04,3.74,1.75,28.06.2018
2,England,Belgium,0,1,2.46,2.83,3.60,28.06.2018
3,Panama,Tunisia,1,2,4.32,3.69,1.86,28.06.2018
4,Mexico,Sweden,0,3,2.22,3.36,3.46,27.06.2018
5,South Korea,Germany,2,0,18.60,8.02,1.16,27.06.2018
6,Serbia,Brazil,0,2,7.20,4.59,1.47,27.06.2018
7,Switzerland,Costa Rica,2,2,1.60,3.65,7.25,27.06.2018
8,Iceland,Croatia,1,2,4.31,3.87,1.83,26.06.2018
9,Nigeria,Argentina,1,2,5.84,5.06,1.49,26.06.2018


In [77]:
pages = [r.get(urls_wc14, params={"stage": "61tCiOIs"}).text, r.get(urls_wc14).text]

In [78]:
create_dataframe(pages)

,team_home,team_away,goals_home,goals_away,odds_home,odds_draw,odds_away,date
0,Algeria,Russia,1,1,3.62,3.64,2.02,26.06.2014
1,South Korea,Belgium,0,1,4.54,3.54,1.84,26.06.2014
2,Portugal,Ghana,2,1,1.77,4.77,3.71,26.06.2014
3,USA,Germany,0,1,8.03,3.05,1.67,26.06.2014
4,Bosnia & Herzegovina,Iran,3,1,2.21,3.51,3.25,25.06.2014
5,Nigeria,Argentina,2,3,9.26,4.15,1.43,25.06.2014
6,Ecuador,France,0,0,5.50,4.09,1.61,25.06.2014
7,Honduras,Switzerland,0,3,6.95,4.40,1.49,25.06.2014
8,Costa Rica,England,0,0,4.39,3.92,1.77,24.06.2014
9,Italy,Uruguay,0,1,2.54,3.44,2.76,24.06.2014
